In [ ]:
!pip install tesseract-ocr
!pip install openpyxl
!pip install Pillow

In [1]:
# 操作 browser 的 API
from selenium import webdriver

# 定義 user-agent
from selenium.webdriver.chrome.options import Options

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# delete 鍵
from selenium.webdriver.common.keys import Keys

# 強制等待 (執行期間休息一下)
from time import sleep

import random

# 整理 json 使用的工具
import json

from selenium.common.exceptions import NoSuchWindowException

# 處理驗證碼
from PIL import Image
import pytesseract

# excel
from openpyxl import load_workbook
from openpyxl import Workbook

In [2]:
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
#options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36")
driver = webdriver.Chrome( options = options )

In [ ]:
# 放置爬取的資料
listData = []
dictionary={}
listCertificate = []
position = 2
mainSpace =None
subSpace=None
handles=[]
# 讀取已存在的 excel 檔案
FilePath = "address_certificate.xlsx"   
workbook = load_workbook(filename = FilePath)

# 取得主要的 sheet
worksheet = workbook['addr'] 

# 從其他網頁得知全部縣市的所有連結
def getSubLink():
    global listData
    driver.get('https://zip5.5432.tw/cityzip/%E6%96%B0%E5%8C%97%E5%B8%82')
    try:
        cssSelector="#zip-table2 > tbody > tr"
        
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, cssSelector)))
        myList=driver.find_elements(by=By.CSS_SELECTOR,value=cssSelector)

        for i in range(1,len(myList)+1):
            cssSelector=("#zip-table2 > tbody > tr:nth-child(%d) > td.zip-area > a" % i)
            target = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
            # 存所有新北市區域連結
            if target.get_attribute('href') not in listData:
                listData.append(target.get_attribute('href'))
    except TimeoutException:
        print('等待逾時！')

# 從其他網頁得知縣市與路名的對應，並存成字典            
def parseSubContents():
    try:
        for i in listData:
            driver.get(i)
            area = driver.find_element(by=By.CSS_SELECTOR,value="#zip-title > h2").text
            # 取得表格所有路名內容
            cssSelector="#zip-table > tbody > tr"
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, cssSelector)))
            myList=driver.find_elements(by=By.CSS_SELECTOR,value=cssSelector)
            subList=[]
            for i in range(1,len(myList)+1):
                cssSelector=("#zip-table > tbody > tr:nth-child(%d) > td" % i)
                mySubList=driver.find_elements(by=By.CSS_SELECTOR,value=cssSelector)
                for j in range(1,len(mySubList)+1):
                    cssSelector=("#zip-table > tbody > tr:nth-child(%d) > td:nth-child(%d)" % (i,j))
                    target = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
                    # 只存大的路名街名 不存巷、弄、段
                    bigRoad = target.text#.split("區")[0]
                    if bigRoad[0] not in ["路","街","道","區"]:
                        Road = bigRoad.split("路")[0].split("街")[0].split("道")[0].split("區")[0]
                    # 如果不以數字開頭，就用數字去切
                    try:
                        if Road[0] not in ["一","二","三","四","五","六","七","八","九","十","南","北"]:
                            road = Road.split("一")[0].split("二")[0].split("三")[0].split("四")[0].split("五")[0].split("六")[0].split("七")[0].split("八")[0].split("九")[0].split("十")[0].split("南")[0].split("北")[0]
                        else:
                            road = Road
                    except:
                        print(bigRoad)
                    # 如果沒有在 list 裡面才加進去
                    if road not in subList:
                        subList.append(road)
            dictionary[area]=subList
    
        # save area and road mapping as a file
        path = 'outputMapping.json'
        # 將字典變成json檔
        jsonString = json.dumps(dictionary, ensure_ascii=False)#重點就是這一個參數，程式碼就不會變亂碼
        with open(path, 'w',encoding="utf-8") as f:
            f.write(jsonString)    
    except TimeoutException:
        print('等待逾時！')  

In [16]:
def mainSetting():
    global mainSpace
    global subSpace
    global handles

    driver.get('https://building-management.publicwork.ntpc.gov.tw/bm_query.jsp?rt=3')
    sleep(3)
    # 輸出當前視窗控制代碼
    mainSpace = driver.current_window_handle

    handles = driver.window_handles
    
    # 建築地址縣市選取按鈕
    cssSelector="#D1V"
    # 按下縣市區域查詢按鈕
    inputCountyBox=driver.find_element(by=By.CSS_SELECTOR,value=cssSelector).click()
    # 獲取當前視窗控制代碼集合，有新視窗（列表型別）
    handles = driver.window_handles
    # 獲取次要視窗
    subSpace = handles[1]

    driver.switch_to.window(subSpace)
    return driver

# 走訪主頁面
def getMainLink(driver):
    #讀出 JSON file文件裡的 area 資訊
    f = open ('outputMapping.json', "r",encoding="utf-8")
    # Reading from file
    json_data = json.loads(f.read())
   
    mylist=[]
    cssSelector="body > div"
    mylst=driver.find_elements(by=By.CSS_SELECTOR,value=cssSelector)
    # 找出次要視窗符合檔案 area 的按鈕
    for j in range(2,len(mylst)+1):# 新北的不同區域  
        print("j= ", j)
        #cnt=0
        cssSelector=("body > div:nth-child(%d)" % j)
        try:
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, cssSelector)))
            target = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
            myTemp=target.text
   
            for i in json_data:
                # 比對爬下來的 json檔 縣市區域
                if myTemp== str(i):
                    for road in json_data[i]:
                        # 選擇次要視窗的區域
                        target = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
                        target.click()  
                        # 跳回主要視窗
                        driver.switch_to.window(mainSpace)
                        # 查詢 "輸入路名" 的 textbox
                        roadbox=driver.find_element(by=By.ID,value="D3")

                        # 輸入之前先清空
                        roadbox.send_keys(Keys.CONTROL+'a')  #全選
                        sleep(2) 
                        roadbox.send_keys(Keys.DELETE) # 刪除
                        sleep(3)
                        print(road)
                        try:
                            roadbox.send_keys(road)
                        except:
                            continue
                        # 圖形驗證碼
                        getNumPic(driver)
                        # 爬表格
                        getContent(driver,myTemp)
                        driver = mainSetting()
        except TimeoutException:
            driver.get('https://building-management.publicwork.ntpc.gov.tw/bm_query.jsp?rt=3')
            pass
            print('等待逾時！') 

In [ ]:
# 取得驗證碼，輸入後按送出
def getNumPic(driver):
    offset = 1500
    # 捲動的 js code
    js_scroll = "(function (){{window.scrollTo({{top:\
    {}, behavior: 'smooth' }});}})();".format(offset)

     # 執行 js code
    driver.execute_script(js_scroll)
    
    # 指定 tesseract 執行檔路徑
    pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
     # 強制等待
    sleep(2)
    
    # 截圖辨識驗證碼
    CoreTry(driver)
    
    # 驗證碼輸錯的警告視窗   
    for i in range(10):  
        try:
            # 若有錯誤視窗
            WebDriverWait(driver, 1).until(EC.alert_is_present())
            alert = driver.switch_to.alert
            alert.accept()
            
            # 重新產生驗證碼
            btnRegen = driver.find_element_by_id("rb_code")
            btnRegen.click()
            # 驗證碼輸入 textbox 清空
            element = driver.find_element_by_id("Z1")
            element.send_keys(Keys.CONTROL+'a')  #全選
            element.send_keys(Keys.DELETE) 
            CoreTry(driver)
        except TimeoutException:
            print("圖片驗證碼正確")
            break
    sleep(3)

def CoreTry(driver):
    offset = 1500
    driver.save_screenshot('test.jpg')
    element = driver.find_element(by=By.ID,value="codeimg")
    left = element.location['x']
    right = element.location['x']+element.size['width']
    top = element.location['y']-offset/2+element.size['height']+20
    bottom = element.location['y']+2*element.size['height']-offset/2+20
    img = Image.open('test.jpg')
    # 擷取驗證碼圖片
    img = img.crop((left, top, right, bottom))

    img = img.convert("L")  # 處理灰白
    
    pixels = img.load()
    for x in range(img.width):
        for y in range(img.height):
            if pixels[x, y] > 130:
                pixels[x, y] = 255
            else:
                pixels[x, y] = 0
    
    result = pytesseract.image_to_string(img)
    print(result)
    # 驗證碼輸入 textbox
    element = driver.find_element(by=By.ID,value="Z1")
    element.send_keys(result)
    selectbtn=driver.find_element_by_css_selector("#form_bm > div.contentform > table > tbody > tr > td:nth-child(2) > button")
    selectbtn.click()
    sleep(5)

In [4]:
# 走訪表格
def getContent(driver,county):
    global position
    global listCertificate
    global worksheet  
    
    # 自訂清單 dictionaries in list
    listCertificate = []

    # 共有幾筆資料
    total = driver.find_element(by=By.ID,value="RecCT")
    print("共有%s筆資料" % total.text)
    totalRow = int(total.text)
    # 若有資料
    if totalRow :
        # 看要翻幾頁
        page = totalRow%7
        
        if page !=0:
            page = totalRow//7 +1
        else:
            page = totalRow//7  
        for i in range(page):
            cssSelector = "#DataBlock > tr"
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, cssSelector)))
            target = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
            # 看有幾列
            myList=driver.find_elements(by=By.CSS_SELECTOR,value=cssSelector)
            # 跑列的迴圈
            for j in range(1,len(myList)+1):
                myRow={}

                # 6 columns
                for k in range(1,7):
                    if k == 1:
                        cssSelector = ("#DataBlock > tr:nth-child(%d) > td:nth-child(1) > a" % j)
                    else:
                        cssSelector = ("#DataBlock > tr:nth-child(%d) > td:nth-child(%d)" % (j,k))                    

                    content = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
                    txt =content.text

                    myRow[str(k)]=txt
                    
                listCertificate.append(myRow)
            # next page button，最後一頁不用按
            if i != (page-1):
                cssSelector ="#DataButton > tr > td > b > a:nth-child(3)"
                nextpage = driver.find_element(by=By.CSS_SELECTOR,value=cssSelector)
                nextpage.click()
            sleep(2)
    else:
        print("沒資料")
    sleep(3)
    
    # 寫到 excel   
    toExcel(county)

def toExcel(county):
    global position
    global listCertificate
    global worksheet
    global workbook
    
    for cert in listCertificate:
        worksheet['A' + str(position)] = county
        worksheet['B' + str(position)] = cert["1"]
        worksheet['C' + str(position)] = cert["2"]
        worksheet['D' + str(position)] = cert["3"]
        worksheet['E' + str(position)] = cert["4"]
        worksheet['F' + str(position)] = cert["5"]
    #if cert["6"]:
        worksheet['G' + str(position)] = cert["6"]
        position += 1 
    # 儲存 workbook
    workbook.save(FilePath)  
    # 關閉 workbook
    workbook.close()

In [17]:
    # 主程式
if __name__ == '__main__':
     #getSubLink()
     #parseSubContents()
     getMainLink(mainSetting())

j=  28
復興
9809



C:\Users\USER\AppData\Local\Temp/ipykernel_11404/982461823.py:144: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  selectbtn=driver.find_element_by_css_selector("#form_bm > div.contentform > table > tbody > tr > td:nth-child(2) > button")


圖片驗證碼正確
共有141筆資料
忠孝
8770

圖片驗證碼正確
共有6筆資料
愛國
6524

圖片驗證碼正確
共有3筆資料
成福
4387

圖片驗證碼正確
共有68筆資料
插角
1432



C:\Users\USER\AppData\Local\Temp/ipykernel_11404/982461823.py:105: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btnRegen = driver.find_element_by_id("rb_code")
C:\Users\USER\AppData\Local\Temp/ipykernel_11404/982461823.py:108: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("Z1")


1629

3226

圖片驗證碼正確
共有14筆資料
文化
9339

2694

1828

3826

圖片驗證碼正確
共有109筆資料
新興
0619

圖片驗證碼正確
共有3筆資料
有木
3080

圖片驗證碼正確
共有9筆資料
東眼
0534

圖片驗證碼正確
共有3筆資料
柑樹
1988

1823

4112

8673

圖片驗證碼正確
共有0筆資料
沒資料
樟坑湖
3892

A197

8830

圖片驗證碼正確
共有0筆資料
沒資料
橫溪
1326

圖片驗證碼正確
共有12筆資料
正義
6096

圖片驗證碼正確
共有12筆資料
民族
1665

2461

6055

圖片驗證碼正確
共有52筆資料
民權
3471

圖片驗證碼正確
共有30筆資料
民生
9498

9892

3973

圖片驗證碼正確
共有127筆資料
永安
0150

圖片驗證碼正確
共有43筆資料
添福
6015

圖片驗證碼正確
共有42筆資料
清水
L077

3290

圖片驗證碼正確
共有4筆資料
溪
1993

2312

圖片驗證碼正確
共有233筆資料
溪東
1281

9337

9396

圖片驗證碼正確
共有108筆資料
牛角坑
1307

8159

圖片驗證碼正確
共有0筆資料
沒資料
白雞
1773

圖片驗證碼正確
共有107筆資料
秀川
1663

圖片驗證碼正確
共有3筆資料
竹安
4786

圖片驗證碼正確
共有0筆資料
沒資料
竹崙
1969

6365

圖片驗證碼正確
共有15筆資料
紫和
0047

圖片驗證碼正確
共有0筆資料
沒資料
紫微
6348

圖片驗證碼正確
共有33筆資料
紫新
3192

圖片驗證碼正確
共有1筆資料
茅埔
1346

9343

1316

0062

圖片驗證碼正確
共有5筆資料
詩朗
8586

圖片驗證碼正確
共有2筆資料
金敏
0325

圖片驗證碼正確
共有0筆資料
沒資料
長泰
8192

圖片驗證碼正確
共有45筆資料
長福
8123

圖片驗證碼正確
共有3筆資料
隆恩
6828

圖片驗證碼正確
共有75筆資料
頭寮坑
1624

1633

圖片驗證碼正確
共有

In [18]:
workbook.close()